## A Root certificate

For our practical example we are going to need some basic pieces. 

First we need to have a root certificate. This practical example would be a valid way to create a local "breakglass" account that requires a proof of possession using a yubikey or similar device. For this reason we will not be setting up a full Certificate Authority, but many of the principles are the same. 

_Most of the important parts of the CA, such as OCSP, would be needed to scale this up to something more then a single system._

_In some steps we will be doing some specifics for the Jupyter Notebook, such as cleanup and placing files to use that could alternately be interactive. I will try and document those as we go along._

_Just as a clarification, these examples work, but they are not necessarily the most efficient. Many of the steps may be able to be rolled up into a single step, configuration files may be able to be abandoned in favor of running everything on the command line, and some of the configurations may have extrenuanous flags and markers that could be weeded out._

In [ ]:
%%bash
# make sure we have a location for our work, this will will be in the git ignore file
if ! [ -d certificates ]; then mkdir certificates; fi
# make a simple passphrase file for the root CA, this is for later to bypass the interactive part
# (that way it will work in the Notebook) 
echo secret > certificates/passphrase

We will start with generating a key. 

_The passphase used is not good, strong passphrases should be used, additionally while we are using a file to set the passphrase on the key, better security should be used in protecting that or destroy it. You could also just use the interactive method by leaving out the `-passout` flag_

__This is the private part of the certificate pair and this needs to be kept under full paranoid "don't let anyone get it" security. If the key and the passphrase were to be compromised, that would be really bad.__

In [ ]:
%%bash
openssl genrsa -aes256 -passout file:certificates/passphrase -out certificates/ca-root.key 2048 

Next we are going to set up an openssl configuration file for the purpose of creating the certificates.

The `req` section sets up the defaults we are using for our certificates. In this section we are turning off prompts, so if we make a mistake and miss an important value, we may end up with a failed certificate.

The `req_distinguished_name` section sets up the certificate properties, if this section was not dictated it would prompt for an interactive answering of the properties. Note that many other values could be added to this section, the ones shown are a minimal set.

The `x509_extensions` section sets up the use parameters of the certificates, for our simple root certificate we will want the `CA:true` and be able to sign other keys with `keyCertSign` and sign relative certificate requests with the `cRLSign` options.

_All of these elements could potentially be added on the command line to generate the certificate, but using a configuration file we can set up all the attributes and locations of files in a more readable way._

In [ ]:
%%bash
cat > certificates/ca-root.cnf << END_OF_FILE
[ req ]
default_bits = 2048
default_md = sha256
default_keyfile = ca-root.key
distinguished_name = req_distinguished_name
#attributes = req_attributes
x509_extensions = v3_ca 
string_mask = utf8only
prompt = no

[ req_distinguished_name ]
C = US
ST = Ohio
L = Cleveland
O = Example
OU = LinuxFest
CN = example.com
emailAddress = lf@example.com

[ v3_ca ]
subjectKeyIdentifier=hash
basicConstraints = critical, CA:true
keyUsage = keyCertSign, cRLSign
END_OF_FILE

Now we can generate our root certificate. Once again we use the passphrase, this time passing it in to access the key, instead of the interactive prompt. 

__We are setting the days to 1, meaning this certificate will expire tomorrow. Just a safety measure, this could potentially be 20 years__

In [ ]:
%%bash
openssl req -x509 -new -nodes -key certificates/ca-root.key -sha256 -days 1 -passin file:certificates/passphrase -out certificates/ca-root.pem -config certificates/ca-root.cnf

Now we can take a look at the certificates directory to see our certificates have been created.

In [ ]:
%%bash 
ls -l certificates

Inspecting the certificate generated, again passing in the passphrase from file instead of interactive prompting.

In [ ]:
%%bash
openssl x509 -passin file:certificates/passphrase -in certificates/ca-root.pem -noout -text

## Basic server certificate

In this section we will build a general server ssl certificate, use it to run a session of this notebook remotely over TTPS.

First, set up so we dont have an interactive prompt.

In [ ]:
%%bash
echo secret > certificates/server.passphrase

Then generate a key.

In [ ]:
%%bash
openssl genrsa -passout file:certificates/server.passphrase -out certificates/server.key 2048

Create a config file for the certificate request.

In [ ]:
%%bash
cat > certificates/server.cnf << END_OF_FILE
[ req ]
default_bits = 2048
default_md = sha256
default_keyfile = privkey.pem
distinguished_name = req_distinguished_name
#attributes = req_attributes
x509_extensions = v3_x509
string_mask = utf8only
prompt = no

[ req_distinguished_name ]
countryName = "US"
stateOrProvinceName = "OH"
localityName = "Columbus"
organizationName = "Ohio Linux Fest 2019"
organizationalUnitName = "Linu PKI System Parts"
commonName = "debian-buster-server.example.com"

[ v3_x509 ]
basicConstraints = critical, CA:FALSE
keyUsage = critical,digitalSignature,keyEncipherment
extendedKeyUsage = serverAuth,clientAuth
subjectKeyIdentifier = hash
END_OF_FILE

Generate a generic certificate request.

In [ ]:
%%bash
openssl req -passin file:certificates/server.passphrase -sha256 -new -key certificates/server.key -nodes -out certificates/server.csr -config certificates/server.cnf

In order to use the root certificate to generate the server certificate.

In [ ]:
%%bash
cat > certificates/ca-server.cnf << END_OF_FILE
[ ca ]
default_ca = my_ca

[ my_ca ]
serial = ./certificates/serial
database = ./certificates/index.txt
new_certs_dir = ./certificates
certificate = ./certificates/ca-root.pem
private_key = ./certificates/ca-root.key
default_md = sha256
default_days = 1
x509_extensions = v3_x509
policy = server_dn 

[ server_dn ]
countryName = supplied
stateOrProvinceName = supplied
localityName = supplied
organizationName = supplied
organizationalUnitName = supplied
commonName = supplied

[ v3_x509 ]
basicConstraints = critical, CA:FALSE
keyUsage = critical,digitalSignature,keyEncipherment
extendedKeyUsage = serverAuth,clientAuth
subjectKeyIdentifier = hash
END_OF_FILE

To create the certificate we need to make sure we have an indexing and serial number, in our case we are just going to make a random serial number.

In [ ]:
%%bash
if ! [ -f certificates/index.txt ]; then touch certificates/index.txt; fi
if ! [ -f certificates/serial ]; then touch certificates/serial; fi
echo $(( $RANDOM % 100 )) > certificates/serial

Now we can generate the certificate from the request.

In [ ]:
%%bash
openssl ca -batch -passin file:certificates/passphrase -config certificates/ca-server.cnf -out certificates/server.crt -infiles certificates/server.csr

Looking at the certificate.

In [ ]:
%%bash
openssl x509 -in certificates/server.crt -noout -text

Sets set our notebook to use SSL. To start we need to grab the root public certificate. 

Navigate to http://debian-buster-server.example.com:8888/edit/certificates/ca-root.pem and select Edit > Download. Save file. 

Since this is outside of the notebook, stop the current notebook.

Edit and copy over the second_jupyter_notebook_config.py to `~/.jupyter/jupyter_notebook_config.py`

Start the notebook. and redirect the browser now to link on token.

__Note: Because we don't have OCSP, the browser may not totally accept the root certificate, however, we should be able to get the certificate to be accepted by Firefox with standard build. On the redirect we will get the error for unknown certificate, to fix that load the ca-root.pem file into the browser certificate roots.__

## Identity Certificate 

With a root certificate that is able to sign, we could potentially make any number of relative certificates associated to it. While creating a simple web SSL certificate would be something we could do, I have decided we will delve a little into making an identity certificate for authentication that we can use with a Yubikey. 

To start out we will make a passphrase for our new key, same as the root certificate this is so we don't get interactive prompts. 

In [ ]:
%%bash
echo secret > certificates/identity.passphrase

We will generate a private key for our new identity.

In [ ]:
%%bash
openssl genrsa -passout file:certificates/identity.passphrase -out certificates/rando.key 2048

For the new identity certificate we will need to make a certificate request to the root certificate, for that we will make a new configuration.

In the `req_distinguished_name` section we now set the properties of the user identity. 

In the `v3_logon_cert` section (which is the `x509_extensions`) we are making it non-authoritative and giving it attributes for usage. The `clientAuth` and `msSmartcardLogin` are elements needed to use the PIV Yubikey mechanism (emulate smartcard).

_In this case we supply a request with distinguished name, different from the server example that was enforced by the ca configuration._

In [ ]:
%%bash
cat > certificates/identity.cnf << END_OF_FILE
[ req ]
default_bits = 2048
default_md = sha256
default_keyfile = privkey.pem
distinguished_name = req_distinguished_name
#attributes = req_attributes
x509_extensions = v3_logon_cert
string_mask = utf8only
prompt = no

[ req_distinguished_name ]
C = US
O = Example
OU = LinuxFest
OU = People
CN = Random Person
UID = rando

[ v3_logon_cert ]
basicConstraints = critical, CA:FALSE
keyUsage = critical, nonRepudiation, digitalSignature, keyEncipherment
extendedKeyUsage = critical, clientAuth, emailProtection, msSmartcardLogin
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid,issuer
END_OF_FILE


Now we can generate a certificate request. 

In [ ]:
%%bash
openssl req -passin file:certificates/identity.passphrase -sha256 -new -config certificates/identity.cnf -key certificates/rando.key -nodes -out certificates/rando.csr

In order to use the root certificate to generate the identity certificate, we need another configuration file (again, this could likely all be done on the command line)

In [ ]:
%%bash
cat > certificates/ca-identity.cnf << END_OF_FILE
[ ca ]
default_ca = my_ca

[ my_ca ]
serial = ./certificates/serial
database = ./certificates/index.txt
new_certs_dir = ./certificates
certificate = ./certificates/ca-root.pem
private_key = ./certificates/ca-root.key
default_md = sha256
default_days = 1
x509_extensions = v3_x509
policy = my_policy

[ my_policy ]
C = supplied
O = supplied
OU = supplied
CN = supplied
OU = supplied
UID = supplied

[ v3_x509 ]
basicConstraints = critical, CA:FALSE
keyUsage = critical, nonRepudiation, digitalSignature, keyEncipherment
extendedKeyUsage = critical, clientAuth, emailProtection, msSmartcardLogin
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid,issuer
END_OF_FILE

Now we can generate the certificate from the request. 

Note that we will use the same index and serial number files as the server certificate, this could be seperated. 

In [ ]:
%%bash
openssl ca -batch -passin file:certificates/passphrase -config certificates/ca-identity.cnf -out certificates/rando.crt -infiles certificates/rando.csr

Looking at the certificate.

In [ ]:
%%bash
openssl x509 -in certificates/rando.crt -noout -text